# Mini projet 2 (poké api)

In [28]:
#!pip install aiocache

In [87]:
import getpass
import requests
import keyring
import time
import asyncio
import aiohttp
from aiohttp import ClientSession
from aiocache import cached, Cache
from aiocache.serializers import JsonSerializer
import cachetools.func # garder le résultat de fetch_with_cache avec les mêmes paramètres

In [76]:
id = 'pikachu'
poke_infos = requests.get(f"https://pokeapi.co/api/v2/pokemon/{id}")
poke_infos.json()

{'abilities': [{'ability': {'name': 'static',
    'url': 'https://pokeapi.co/api/v2/ability/9/'},
   'is_hidden': False,
   'slot': 1},
  {'ability': {'name': 'lightning-rod',
    'url': 'https://pokeapi.co/api/v2/ability/31/'},
   'is_hidden': True,
   'slot': 3}],
 'base_experience': 112,
 'cries': {'latest': 'https://raw.githubusercontent.com/PokeAPI/cries/main/cries/pokemon/latest/25.ogg',
  'legacy': 'https://raw.githubusercontent.com/PokeAPI/cries/main/cries/pokemon/legacy/25.ogg'},
 'forms': [{'name': 'pikachu',
   'url': 'https://pokeapi.co/api/v2/pokemon-form/25/'}],
 'game_indices': [{'game_index': 84,
   'version': {'name': 'red', 'url': 'https://pokeapi.co/api/v2/version/1/'}},
  {'game_index': 84,
   'version': {'name': 'blue', 'url': 'https://pokeapi.co/api/v2/version/2/'}},
  {'game_index': 84,
   'version': {'name': 'yellow',
    'url': 'https://pokeapi.co/api/v2/version/3/'}},
  {'game_index': 25,
   'version': {'name': 'gold', 'url': 'https://pokeapi.co/api/v2/version

In [78]:
type = 'water'
poke_infos = requests.get(f"https://pokeapi.co/api/v2/type/{type}")
poke_infos.json()

{'damage_relations': {'double_damage_from': [{'name': 'grass',
    'url': 'https://pokeapi.co/api/v2/type/12/'},
   {'name': 'electric', 'url': 'https://pokeapi.co/api/v2/type/13/'}],
  'double_damage_to': [{'name': 'ground',
    'url': 'https://pokeapi.co/api/v2/type/5/'},
   {'name': 'rock', 'url': 'https://pokeapi.co/api/v2/type/6/'},
   {'name': 'fire', 'url': 'https://pokeapi.co/api/v2/type/10/'}],
  'half_damage_from': [{'name': 'steel',
    'url': 'https://pokeapi.co/api/v2/type/9/'},
   {'name': 'fire', 'url': 'https://pokeapi.co/api/v2/type/10/'},
   {'name': 'water', 'url': 'https://pokeapi.co/api/v2/type/11/'},
   {'name': 'ice', 'url': 'https://pokeapi.co/api/v2/type/15/'}],
  'half_damage_to': [{'name': 'water',
    'url': 'https://pokeapi.co/api/v2/type/11/'},
   {'name': 'grass', 'url': 'https://pokeapi.co/api/v2/type/12/'},
   {'name': 'dragon', 'url': 'https://pokeapi.co/api/v2/type/16/'}],
  'no_damage_from': [],
  'no_damage_to': []},
 'game_indices': [{'game_index':

## Écrivez une fonction qui demande au user : 

Le nom/id d'un Pokémon et affiche ses statistiques : points de vie, attaque, défense, etc. (Ne retourner pas tout simplement le grand json)

Le nom de deux Pokémon et compare leurs statistiques. Lequel est le plus fort (en points de vie et en attaque) ?

Le type des Pokémon (fire, dragon, etc.) et calcule le nombre des Pokémon de ce type fire, la moyenne des points de vie (HP)

In [80]:
def get_pokemon_data(pokemon_name_or_id):
    url = f"https://pokeapi.co/api/v2/pokemon/{pokemon_name_or_id.lower()}"
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Erreur : Le Pokémon {pokemon_name_or_id} n'a pas été trouvé.")
        return None

def display_pokemon_stats(pokemon_name_or_id):
    data = get_pokemon_data(pokemon_name_or_id)
    if data:
        name = data['name']
        hp = data['stats'][0]['base_stat']
        attack = data['stats'][1]['base_stat']
        defense = data['stats'][2]['base_stat']
        speed = data['stats'][5]['base_stat']
        print(f"\nStatistiques de {name.capitalize()}:")
        print(f"- Points de vie (HP) : {hp}")
        print(f"- Attaque : {attack}")
        print(f"- Défense : {defense}")
        print(f"- Vitesse : {speed}")

def compare_pokemon(pokemon1, pokemon2):
    data1 = get_pokemon_data(pokemon1)
    data2 = get_pokemon_data(pokemon2)
    if data1 and data2:
        hp1 = data1['stats'][0]['base_stat']
        attack1 = data1['stats'][1]['base_stat']
        hp2 = data2['stats'][0]['base_stat']
        attack2 = data2['stats'][1]['base_stat']
        
        print(f"\nComparaison entre {pokemon1.capitalize()} et {pokemon2.capitalize()}:")
        if hp1 > hp2:
            print(f"{pokemon1.capitalize()} a plus de points de vie ({hp1} contre {hp2}).")
        else:
            print(f"{pokemon2.capitalize()} a plus de points de vie ({hp2} contre {hp1}).")
        
        if attack1 > attack2:
            print(f"{pokemon1.capitalize()} a une meilleure attaque ({attack1} contre {attack2}).")
        else:
            print(f"{pokemon2.capitalize()} a une meilleure attaque ({attack2} contre {attack1}).")

def calculate_type_stats(pokemon_type):
    url = f"https://pokeapi.co/api/v2/type/{pokemon_type.lower()}"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        pokemon_list = data['pokemon']
        total_hp = 0
        count = 0

        for pokemon_entry in pokemon_list:
            pokemon_name = pokemon_entry['pokemon']['name']
            pokemon_data = get_pokemon_data(pokemon_name)
            if pokemon_data:
                hp = pokemon_data['stats'][0]['base_stat']
                total_hp += hp
                count += 1

        if count > 0:
            average_hp = total_hp / count
            print(f"\nStatistiques pour le type {pokemon_type.capitalize()}:")
            print(f"- Nombre de Pokémon : {count}")
            print(f"- Moyenne des points de vie (HP) : {average_hp:.2f}")
        else:
            print(f"Aucun Pokémon trouvé pour le type {pokemon_type}.")
    else:
        print(f"Erreur : Le type {pokemon_type} n'a pas été trouvé.")

# Exemple d'utilisation
while True:
    print("\nChoisissez une option:")
    print("1. Afficher les statistiques d'un Pokémon")
    print("2. Comparer deux Pokémon")
    print("3. Calculer les statistiques pour un type de Pokémon")
    print("4. Quitter")
    
    choice = input("Entrez votre choix (1-4): ")
    
    if choice == '1':
        pokemon_name = input("Entrez le nom ou l'ID du Pokémon: ")
        display_pokemon_stats(pokemon_name)
    elif choice == '2':
        pokemon1 = input("Entrez le nom ou l'ID du premier Pokémon: ")
        pokemon2 = input("Entrez le nom ou l'ID du deuxième Pokémon: ")
        compare_pokemon(pokemon1, pokemon2)
    elif choice == '3':
        pokemon_type = input("Entrez le type de Pokémon (ex: fire, water, grass): ")
        calculate_type_stats(pokemon_type)
    elif choice == '4':
        print("Au revoir!")
        break
    else:
        print("Choix invalide, veuillez réessayer.")


Choisissez une option:
1. Afficher les statistiques d'un Pokémon
2. Comparer deux Pokémon
3. Calculer les statistiques pour un type de Pokémon
4. Quitter


Entrez votre choix (1-4):  1
Entrez le nom ou l'ID du Pokémon:  2



Statistiques de Ivysaur:
- Points de vie (HP) : 60
- Attaque : 62
- Défense : 63
- Vitesse : 60

Choisissez une option:
1. Afficher les statistiques d'un Pokémon
2. Comparer deux Pokémon
3. Calculer les statistiques pour un type de Pokémon
4. Quitter


Entrez votre choix (1-4):  2
Entrez le nom ou l'ID du premier Pokémon:  squirtle
Entrez le nom ou l'ID du deuxième Pokémon:  charizard



Comparaison entre Squirtle et Charizard:
Charizard a plus de points de vie (78 contre 44).
Charizard a une meilleure attaque (84 contre 48).

Choisissez une option:
1. Afficher les statistiques d'un Pokémon
2. Comparer deux Pokémon
3. Calculer les statistiques pour un type de Pokémon
4. Quitter


Entrez votre choix (1-4):  3
Entrez le type de Pokémon (ex: fire, water, grass):  fire



Statistiques pour le type Fire:
- Nombre de Pokémon : 103
- Moyenne des points de vie (HP) : 71.99

Choisissez une option:
1. Afficher les statistiques d'un Pokémon
2. Comparer deux Pokémon
3. Calculer les statistiques pour un type de Pokémon
4. Quitter


Entrez votre choix (1-4):  4


Au revoir!


## Q1 commune :

Proposer une fonction qui permet gérer les erreurs courantes lors des requêtes vers l’API ? Celle-ci permettra des pauses (time.sleep), des mécanismes de retry (tentatives de nouvelles exécution), … ⇒ des calls plus robustes : 

Par exemple : si le statut 429 (Too Many Requests) est retourné, 500/503 (Internal Server Error / Service Unavailable : erreur de serveur), éventuellement : 400 (Bad Request) , 401 (Unauthorized : token absent ou invalide) , 403 (Forbidden) , 404, identifiant non valide,… ?

In [82]:
def robust_api_call(url, max_retries=5, backoff_factor=2):
    """
    Requête HTTP GET vers l'URL donnée, avec gestion des erreurs.
    
    Paramètres :
    - url (str) : l'URL de l'API à appeler.
    - max_retries (int) : nombre maximum de tentatives en cas d'échec (défaut : 5).
    - backoff_factor (int) : facteur multiplicatif pour le délai d'attente entre les tentatives (défaut : 2).
    
    Retourne :
    - dict : Les données JSON de la réponse si la requête est réussie.
    - None : Si toutes les tentatives échouent.
    """
    retries = 0
    
    while retries < max_retries:
        try:
            response = requests.get(url)
            status_code = response.status_code
            
            # Si la requête est réussie
            if status_code == 200:
                return response.json()
            
            # Gérer les erreurs spécifiques
            if status_code == 429:
                # Trop de requêtes, attendre avant de réessayer
                retry_after = int(response.headers.get("Retry-After", 1))
                print(f"Erreur 429 : Trop de requêtes. Attente de {retry_after} secondes avant de réessayer.")
                time.sleep(retry_after)
            elif status_code in [500, 503]:
                # Erreurs serveur, attendre et réessayer
                wait_time = backoff_factor ** retries # backoff_factor ** retries = Délai augmentant avec le nombre de retry
                print(f"Erreur {status_code} : Problème de serveur. Réessayer dans {wait_time} secondes.")
                time.sleep(wait_time)
            elif status_code == 400:
                print("Erreur 400 : Mauvaise requête. Vérifiez l'URL ou les paramètres.")
                break
            elif status_code == 401:
                print("Erreur 401 : Non autorisé. Vérifiez le token d'authentification.")
                break
            elif status_code == 403:
                print("Erreur 403 : Accès interdit. Vous n'avez pas les droits nécessaires.")
                break
            elif status_code == 404:
                print("Erreur 404 : Ressource non trouvée. L'identifiant est peut-être incorrect.")
                break
            else:
                # Autres erreurs
                print(f"Erreur {status_code} : Une erreur inconnue est survenue.")
                break
            
        except requests.exceptions.ConnectionError:
            print("Erreur de connexion : Impossible de joindre l'API. Réessayer dans quelques secondes.")
            time.sleep(backoff_factor ** retries)
        except requests.exceptions.Timeout:
            print("Délai d'attente dépassé : L'API n'a pas répondu à temps. Réessayer.")
            time.sleep(backoff_factor ** retries)
        except requests.exceptions.RequestException as e:
            # Pour d'autres erreurs non prévues
            print(f"Une erreur inattendue est survenue : {e}")
            break
        
        # Augmenter le nombre de tentatives
        retries += 1
        print(f"Tentative {retries} sur {max_retries} échouée. Nouvelle tentative...")
    
    # Si toutes les tentatives échouent, retourner None
    print("Échec de la requête après plusieurs tentatives.")
    return None

# Exemple d'utilisation
url = "https://pokeapi.co/api/v2/pokemon/pikachu/"  # URL de l'API Pokémon pour pikachu
result = robust_api_call(url)

if result:
    print("Requête réussie :", result)
else:
    print("Impossible de récupérer les données après plusieurs tentatives.")


Requête réussie : {'abilities': [{'ability': {'name': 'static', 'url': 'https://pokeapi.co/api/v2/ability/9/'}, 'is_hidden': False, 'slot': 1}, {'ability': {'name': 'lightning-rod', 'url': 'https://pokeapi.co/api/v2/ability/31/'}, 'is_hidden': True, 'slot': 3}], 'base_experience': 112, 'cries': {'latest': 'https://raw.githubusercontent.com/PokeAPI/cries/main/cries/pokemon/latest/25.ogg', 'legacy': 'https://raw.githubusercontent.com/PokeAPI/cries/main/cries/pokemon/legacy/25.ogg'}, 'forms': [{'name': 'pikachu', 'url': 'https://pokeapi.co/api/v2/pokemon-form/25/'}], 'game_indices': [{'game_index': 84, 'version': {'name': 'red', 'url': 'https://pokeapi.co/api/v2/version/1/'}}, {'game_index': 84, 'version': {'name': 'blue', 'url': 'https://pokeapi.co/api/v2/version/2/'}}, {'game_index': 84, 'version': {'name': 'yellow', 'url': 'https://pokeapi.co/api/v2/version/3/'}}, {'game_index': 25, 'version': {'name': 'gold', 'url': 'https://pokeapi.co/api/v2/version/4/'}}, {'game_index': 25, 'version

## Q2 commune :

Proposer une fonction qui permet de gérer des millions de requêtes de manière efficace et simulez à l’aide d’un script une charge très élevée de call vers plusieurs rootes : ["https://api.example.com/data1", "https://api.example.com/data2", "https://api.example.com/data3"] ? Indice : concurrence et parallélisme en Python, asyncio 

voir q2commune.py

In [84]:
async def fetch(url, session):
    """
    Effectue une requête GET asynchrone à l'URL spécifiée.
    
    Paramètres :
    - url (str) : l'URL de l'API à appeler.
    - session (ClientSession) : la session aiohttp pour réutiliser les connexions.
    
    Retourne :
    - dict : Les données JSON de la réponse si la requête est réussie.
    """
    try:
        async with session.get(url) as response:
            if response.status == 200:
                data = await response.json()
                print(f"Succès : {url}")
                return data
            else:
                print(f"Erreur {response.status} : {url}")
                return None
    except Exception as e:
        print(f"Exception pour {url}: {e}")
        return None

async def fetch_all(urls, max_concurrent_requests):
    """
    Gère l'envoi de requêtes asynchrones à toutes les URLs avec un contrôle de la concurrence.
    
    Paramètres :
    - urls (list) : la liste des URLs à appeler.
    - max_concurrent_requests (int) : nombre maximum de requêtes concurrentes.
    """
    connector = aiohttp.TCPConnector(limit_per_host=max_concurrent_requests)
    async with ClientSession(connector=connector) as session:
        tasks = []
        for url in urls:
            tasks.append(fetch(url, session))
        results = await asyncio.gather(*tasks)
        return results

def simulate_high_load(urls, total_requests, max_concurrent_requests):
    """
    Simule une charge élevée de requêtes en répétant les appels vers les URLs.
    
    Paramètres :
    - urls (list) : la liste des URLs à appeler.
    - total_requests (int) : nombre total de requêtes à envoyer.
    - max_concurrent_requests (int) : nombre maximum de requêtes concurrentes.
    """
    # Générer la liste des URLs à appeler
    all_urls = urls * (total_requests // len(urls))
    start_time = time.time()

    # Exécuter les requêtes asynchrones
    asyncio.run(fetch_all(all_urls, max_concurrent_requests))

    end_time = time.time()
    print(f"Temps total pour {total_requests} requêtes : {end_time - start_time:.2f} secondes")

# Exemple d'utilisation
if __name__ == "__main__":
    urls = [
        "https://pokeapi.co/api/v2/pokemon/pikachu/",
        "https://pokeapi.co/api/v2/pokemon/jigglypuff/",
        "https://pokeapi.co/api/v2/pokemon/charizard/"
    ]
    total_requests = 1000  # Nombre total de requêtes à envoyer (simuler une charge élevée)
    max_concurrent_requests = 100  # Limite de requêtes concurrentes

    simulate_high_load(urls, total_requests, max_concurrent_requests)


RuntimeError: asyncio.run() cannot be called from a running event loop

## Q3 commune :

Avancé - Améliorer encore cette dernière fonction en y spécifiant un cache avec un délai d’expiration d’1 heure ET simulez/montrer son bon fonctionnement ? Cela permet de : 

- Réduire les appels répétitifs à l'API (surtout si on doit retourner les mêmes data via l’AP) et donc d'alléger la charge sur le serveur.
- Améliorer la rapidité de la réponse en se basant sur le cache et donc Améliorer les perf. de l’app. en prod.

voir q3commune.py

In [89]:
@cachetools.func.ttl_cache(maxsize=100, ttl=60 * 60) # durée limitée : 60 * 60 secondes = 1h
async def fetch_with_cache(url, session):
    """
    Effectue une requête GET asynchrone à l'URL spécifiée avec mise en cache.
    
    Paramètres :
    - url (str) : l'URL de l'API à appeler.
    - session (ClientSession) : la session aiohttp pour réutiliser les connexions.
    
    Retourne :
    - dict : Les données JSON de la réponse si la requête est réussie.
    """
    try:
        async with session.get(url) as response:
            if response.status == 200:
                data = await response.json()
                print(f"[CACHE MISS] Requête effectuée : {url}")
                return data
            else:
                print(f"Erreur {response.status} : {url}")
                return None
    except Exception as e:
        print(f"Exception pour {url}: {e}")
        return None

async def fetch_all_with_cache(urls, max_concurrent_requests):
    """
    Gère l'envoi de requêtes asynchrones à toutes les URLs avec un contrôle de la concurrence et du cache.
    
    Paramètres :
    - urls (list) : la liste des URLs à appeler.
    - max_concurrent_requests (int) : nombre maximum de requêtes concurrentes.
    """
    connector = aiohttp.TCPConnector(limit_per_host=max_concurrent_requests)
    async with ClientSession(connector=connector) as session:
        tasks = []
        for url in urls:
            tasks.append(fetch_with_cache(url, session))
        results = await asyncio.gather(*tasks)
        return results

def simulate_high_load_with_cache(urls, total_requests, max_concurrent_requests):
    """
    Simule une charge élevée de requêtes en répétant les appels vers les URLs avec gestion du cache.
    
    Paramètres :
    - urls (list) : la liste des URLs à appeler.
    - total_requests (int) : nombre total de requêtes à envoyer.
    - max_concurrent_requests (int) : nombre maximum de requêtes concurrentes.
    """
    # Générer la liste des URLs à appeler
    all_urls = urls * (total_requests // len(urls))
    start_time = time.time()

    # Exécuter les requêtes asynchrones avec cache
    asyncio.run(fetch_all_with_cache(all_urls, max_concurrent_requests))

    end_time = time.time()
    print(f"Temps total pour {total_requests} requêtes : {end_time - start_time:.2f} secondes")

# Exemple d'utilisation
if __name__ == "__main__":
    urls = [
        "https://pokeapi.co/api/v2/pokemon/pikachu/",
        "https://pokeapi.co/api/v2/pokemon/jigglypuff/",
        "https://pokeapi.co/api/v2/pokemon/charizard/"
    ]
    total_requests = 1000  # Nombre total de requêtes à envoyer (simuler une charge élevée)
    max_concurrent_requests = 100  # Limite de requêtes concurrentes

    simulate_high_load_with_cache(urls, total_requests, max_concurrent_requests)


RuntimeError: asyncio.run() cannot be called from a running event loop

## Simulez un combat entre deux Pokémon en utilisant leurs statistiques de base. Le Pokémon qui inflige le plus de dégâts totaux en cinq tours remporte le combat

### Version sans poké api

In [92]:
class Pokemon:
    def __init__(self, name, hp, attack, defense):
        self.name = name
        self.hp = hp
        self.attack = attack
        self.defense = defense

    def calculate_damage(self, opponent):
        """
        Calcule les dégâts infligés à l'adversaire en fonction de l'attaque et de la défense.
        """
        damage = self.attack - (opponent.defense / 2)
        return max(1, int(damage))  # Les dégâts doivent être au moins 1

def simulate_battle(pokemon1, pokemon2, rounds=5):
    """
    Simule un combat entre deux Pokémon sur un nombre de tours spécifié.
    
    Paramètres :
    - pokemon1 : Premier Pokémon
    - pokemon2 : Deuxième Pokémon
    - rounds : Nombre de tours (5 par défaut)
    
    Retourne le Pokémon vainqueur ou 'Égalité' si les deux Pokémon infligent les mêmes dégâts totaux.
    """
    total_damage1 = 0
    total_damage2 = 0

    for round in range(1, rounds + 1):
        # Pokémon 1 attaque Pokémon 2
        damage_by_pokemon1 = pokemon1.calculate_damage(pokemon2)
        pokemon2.hp -= damage_by_pokemon1
        total_damage1 += damage_by_pokemon1
        print(f"Tour {round}: {pokemon1.name} inflige {damage_by_pokemon1} dégâts à {pokemon2.name} (HP restant: {pokemon2.hp})")
        
        # Vérifier si Pokémon 2 est K.O.
        if pokemon2.hp <= 0:
            print(f"\n{pokemon2.name} est K.O.! {pokemon1.name} remporte le combat !")
            return pokemon1

        # Pokémon 2 attaque Pokémon 1
        damage_by_pokemon2 = pokemon2.calculate_damage(pokemon1)
        pokemon1.hp -= damage_by_pokemon2
        total_damage2 += damage_by_pokemon2
        print(f"Tour {round}: {pokemon2.name} inflige {damage_by_pokemon2} dégâts à {pokemon1.name} (HP restant: {pokemon1.hp})")

        # Vérifier si Pokémon 1 est K.O.
        if pokemon1.hp <= 0:
            print(f"\n{pokemon1.name} est K.O.! {pokemon2.name} remporte le combat !")
            return pokemon2

    # Afficher les dégâts totaux infligés par chaque Pokémon
    print(f"\nDégâts totaux infligés par {pokemon1.name} : {total_damage1}")
    print(f"Dégâts totaux infligés par {pokemon2.name} : {total_damage2}")

    # Déterminer le vainqueur basé sur les dégâts totaux si personne n'est K.O.
    if total_damage1 > total_damage2:
        print(f"\n{pokemon1.name} remporte le combat avec le plus de dégâts infligés !")
        return pokemon1
    elif total_damage2 > total_damage1:
        print(f"\n{pokemon2.name} remporte le combat avec le plus de dégâts infligés !")
        return pokemon2
    else:
        print("\nLe combat se termine par une égalité.")
        return "Égalité"

# Exemple d'utilisation
if __name__ == "__main__":
    # Créer deux Pokémon avec des statistiques fictives
    pikachu = Pokemon(name="Pikachu", hp=35, attack=55, defense=40)
    charmander = Pokemon(name="Charmander", hp=39, attack=52, defense=43)

    # Simuler le combat entre les deux Pokémon
    simulate_battle(pikachu, charmander)


Tour 1: Pikachu inflige 33 dégâts à Charmander (HP restant: 6)
Tour 1: Charmander inflige 32 dégâts à Pikachu (HP restant: 3)
Tour 2: Pikachu inflige 33 dégâts à Charmander (HP restant: -27)

Charmander est K.O.! Pikachu remporte le combat !


### Version avec poké api (calls robustes)

In [94]:
class Pokemon:
    def __init__(self, name):
        self.name = name
        self.hp = 0
        self.attack = 0
        self.defense = 0
        self.get_stats_from_api()

    def get_stats_from_api(self):
        """
        Récupère les statistiques du Pokémon à partir de l'API PokéAPI.
        """
        url = f"https://pokeapi.co/api/v2/pokemon/{self.name.lower()}"
        response = robust_api_call(url)

        data = response
        for stat in data['stats']:
            if stat['stat']['name'] == 'hp':
                self.hp = stat['base_stat']
            elif stat['stat']['name'] == 'attack':
                self.attack = stat['base_stat']
            elif stat['stat']['name'] == 'defense':
                self.defense = stat['base_stat']
        print(f"{self.name} - HP: {self.hp}, Attack: {self.attack}, Defense: {self.defense}")


    def calculate_damage(self, opponent):
        """
        Calcule les dégâts infligés à l'adversaire en fonction de l'attaque et de la défense.
        """
        damage = self.attack - (opponent.defense / 2)
        return max(1, int(damage)) # pour éviter damage <= 0

def simulate_battle(pokemon1, pokemon2, rounds=5):
    """
    Simule un combat entre deux Pokémon sur un nombre de tours spécifié.
    
    Paramètres :
    - pokemon1 : Premier Pokémon
    - pokemon2 : Deuxième Pokémon
    - rounds : Nombre de tours (5 par défaut)
    
    Retourne le Pokémon vainqueur ou 'Égalité' si les deux Pokémon infligent les mêmes dégâts totaux.
    """
    total_damage1 = 0
    total_damage2 = 0

    for round in range(1, rounds + 1):
        # Pokémon 1 attaque Pokémon 2
        damage_by_pokemon1 = pokemon1.calculate_damage(pokemon2)
        pokemon2.hp -= damage_by_pokemon1
        total_damage1 += damage_by_pokemon1
        print(f"Tour {round}: {pokemon1.name} inflige {damage_by_pokemon1} dégâts à {pokemon2.name} (HP restant: {pokemon2.hp})")
        
        # Vérifier si Pokémon 2 est K.O.
        if pokemon2.hp <= 0:
            print(f"\n{pokemon2.name} est K.O.! {pokemon1.name} remporte le combat !")
            return pokemon1

        # Pokémon 2 attaque Pokémon 1
        damage_by_pokemon2 = pokemon2.calculate_damage(pokemon1)
        pokemon1.hp -= damage_by_pokemon2
        total_damage2 += damage_by_pokemon2
        print(f"Tour {round}: {pokemon2.name} inflige {damage_by_pokemon2} dégâts à {pokemon1.name} (HP restant: {pokemon1.hp})")

        # Vérifier si Pokémon 1 est K.O.
        if pokemon1.hp <= 0:
            print(f"\n{pokemon1.name} est K.O.! {pokemon2.name} remporte le combat !")
            return pokemon2

    # Afficher les dégâts totaux infligés par chaque Pokémon
    print(f"\nDégâts totaux infligés par {pokemon1.name} : {total_damage1}")
    print(f"Dégâts totaux infligés par {pokemon2.name} : {total_damage2}")

    # Déterminer le vainqueur basé sur les dégâts totaux si personne n'est K.O.
    if total_damage1 > total_damage2:
        print(f"\n{pokemon1.name} remporte le combat avec le plus de dégâts infligés !")
        return pokemon1
    elif total_damage2 > total_damage1:
        print(f"\n{pokemon2.name} remporte le combat avec le plus de dégâts infligés !")
        return pokemon2
    else:
        print("\nLe combat se termine par une égalité.")
        return "Égalité"

if __name__ == "__main__":

    # Simuler le combat entre les deux Pokémon
    simulate_battle(Pokemon("pikachu"), Pokemon("jigglypuff"))


pikachu - HP: 35, Attack: 55, Defense: 40
jigglypuff - HP: 115, Attack: 45, Defense: 20
Tour 1: pikachu inflige 45 dégâts à jigglypuff (HP restant: 70)
Tour 1: jigglypuff inflige 25 dégâts à pikachu (HP restant: 10)
Tour 2: pikachu inflige 45 dégâts à jigglypuff (HP restant: 25)
Tour 2: jigglypuff inflige 25 dégâts à pikachu (HP restant: -15)

pikachu est K.O.! jigglypuff remporte le combat !
